In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
import time

# 경로는 기존과 동일하게 유지합니다.
bucket = "say1-5team-bucket"
role = get_execution_role()

est = PyTorch(
    entry_point="train-vit.py",          # 아래에서 생성할 ViT 훈련 스크립트 파일명
    source_dir=".",
    role=role,
    framework_version="2.1",
    py_version="py310",
    instance_count=1,
    instance_type="ml.g4dn.2xlarge",      # 16GB VRAM
    hyperparameters={
        "backbone": "vit_b_16",           # 훈련할 모델을 ViT로 변경
        "epochs": 30,
        "freeze-epochs": 2,
        "img-size": 224,                  # ViT 기본 권장 해상도로 변경
        "batch-size": 12,                 # ViT 모델 크기를 고려해 배치 사이즈 조정 (OOM 방지)
        "lr": 3e-4,
        "weight-decay": 1e-4,
        "label-smoothing": 0.05,
        "seed": 42,
    },
    output_path=f"s3://{bucket}/vit-output/", # 출력 경로는 구분을 위해 변경하는 것을 권장
    base_job_name="vit-b16-skin",
)

# 데이터 입력 경로는 기존과 동일합니다.
inputs = {
    "train": TrainingInput(f"s3://{bucket}/densenet-training-data/train"),
    "val":   TrainingInput(f"s3://{bucket}/densenet-training-data/val"),
    "test":  TrainingInput(f"s3://{bucket}/densenet-training-data/test"),
}

# 훈련 직업 이름을 생성하고 훈련을 시작합니다.
job_name = f"vit-b16-skin-job-{time.strftime('%Y-%m-%d-%H-%M-%S')}"
est.fit(inputs, job_name=job_name, logs=True)

print("model_data:", est.model_data)